# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install bs4 
from bs4 import BeautifulSoup
!pip install lxml
import requests
from datetime import datetime
import seaborn as sb
from functools import reduce
from time import sleep
import ast
import urllib
import requests
from bs4 import Comment

C:\Users\Danel\anaconda3\envs\quant\lib\site-packages\numpy\core\__init__.py:27: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Danel\anaconda3\envs\quant\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Danel\anaconda3\envs\quant\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


# Gathering Data

Scrape current baseball statistics from baseball-reference.com

In [2]:
def Web_Scrape_Team_Data(year, teams):

    streak_mapping = {'+': 1, '++': 2, '+++': 3, '++++': 4, '+++++': 5, '++++++':6, '+++++++': 7, '++++++++': 8, '+++++++++': 9, '++++++++++': 10, '+++++++++++': 11, '++++++++++++': 12, '+++++++++++++':13, '++++++++++++++':14, '+++++++++++++++': 15,
                      '-': -1, '--': -2, '---': -3, '----': -4, '-----': -5, '------':-6, '-------': -7, '--------': -8, '---------': -9, '----------': -10, '-----------': -11, '------------': -12, '-------------':-13, '--------------':-14, '---------------': -15}

    teamsData = {}

    for index, team in enumerate(teams):
        ## Collect schedule information for each team
        baseball_schedule_url = 'https://www.baseball-reference.com/teams/' + team + '/' + year + '-schedule-scores.shtml'
        dfs = pd.read_html(baseball_schedule_url)
        scheduleDF = pd.concat(dfs)
        scheduleDF.drop(['Unnamed: 2', 'Unnamed: 4', 'W-L', 'D/N', 'Orig. Scheduled'], axis = 1, inplace = True)
        scheduleDF = scheduleDF[scheduleDF.Opp != 'Opp'] # delete rows where the column statistics are repeated on the website
        scheduleDF.to_csv('1.csv')

        ## Collect batting data for each game every team has played
        baseball_batting_data_url = 'https://www.baseball-reference.com/teams/tgl.cgi?team=' + team + '&t=b&year=2021'
        dfs = pd.read_html(baseball_batting_data_url)
        battingDF = pd.concat(dfs)
        battingDF.drop(['Opp', 'R', 'Date', 'Rk', 'Unnamed: 3', 'Gtm', 'Rslt'], axis = 1, inplace = True)
        battingDF = battingDF[battingDF.PA != 'PA'] # delete rows where the column names are repeated on the website
        battingDF.to_csv('2.csv')

        ## Collect pitching data for each game every team has played
        baseball_pitching_data_url = 'https://www.baseball-reference.com/teams/tgl.cgi?team=' + team + '&t=p&year=2021'
        dfs = pd.read_html(baseball_pitching_data_url)
        PitchingDF = pd.concat(dfs)
        PitchingDF.drop(['Opp', 'R', 'Date', 'Avg', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                         'Other Game Totals', 'Rk', 'Gtm', 'Unnamed: 3', 'Rslt', '#', 'Umpire'], axis = 1, inplace = True)
        PitchingDF = PitchingDF[PitchingDF.ERA != 'ERA'] # delete rows where the column statistics are repeated on the website
        #PitchingDF.drop_duplicates(keep = 'first', inplace = True)
        PitchingDF.to_csv('3.csv')

        # Combining all dataframes into one big dataframe - including schedule, batting, and pitching stats
        CombDF1 = scheduleDF.merge(battingDF, how = 'right', left_index = True, right_index=True)
        #CombDF1 = scheduleDF.merge(battingDF, how = 'right', left_index = True, right_index=True)
        teamsData[team] = CombDF1.merge(PitchingDF, how = 'left', left_index = True, right_index=True, suffixes=('_batting', '_pitching'))
        teamsData[team].drop_duplicates(subset = 'Gm#', keep = 'first', inplace = True) # delete duplicate games (if any)

        # Change Streak Column to numerical values based on streak_mapping dictionary
        teamsData[team]['Streak'] = teamsData[team]['Streak'].map(streak_mapping)

        # Extract Starting Pitcher Info for Home Team
        teamsData[team]['Pitcher'] = teamsData[team]['Pitchers Used (Rest-GameScore-Dec)'].str.extract(r'^([a-zA-Z]{1}[.]{1}[^\d\( ]*)', expand=False)
        #teamsData[team]['Starting_Pitcher_Opp'] = teamsData[team]['Opp. Starter (GmeSc)'].str.extract(r'^([a-zA-Z]{1}[.]{1}[^\d\( ]*)', expand=False)

        # Extract Last Name From Pitchers Name Column - store this to merge dataframes later
        teamsData[team]['Pitcher Last Name'] = teamsData[team]['Pitcher'].str[2:]
        teamsData[team]['Pitcher First Initial'] = teamsData[team]['Pitcher'].str[:1]
        #teamsData[team]['Starting_Pitcher_Last_Name_Opp'] = teamsData[team]['Starting_Pitcher_Opp'].str[2:]

        ## change columns to correct type - ADD TO THIS LATER
        teamsData[team][['R', 'RA', 'cLI', 'PA', 'AB_batting', 'H_batting', '2B_batting', '3B_batting', 'HR_batting', 'RBI',
                        'BB_batting', 'HBP_batting', 'IBB_batting', 'SO_batting', 'SH_batting', 'ROE_batting', 'GDP_batting',
                        'SB_batting', 'CS_batting', 'BA', 'OBP', 'SLG', 'OPS', 'LOB', '#', 'IP', 'H_pitching', 'ER', 'UER', 
                        'BB_pitching', 'SO_pitching', 'HR_pitching', 'HBP_pitching', 'ERA', 'BF', 'Pit', 'Str', 'SB_pitching',
                        'CS_pitching', 'AB_pitching', '2B_pitching', '3B_pitching', 'ROE_pitching', 'GDP_pitching', 'Gm#']] = teamsData[team][['R', 'RA', 'cLI', 'PA', 'AB_batting', 'H_batting', '2B_batting', '3B_batting', 'HR_batting', 'RBI',
                                                                                                                                        'BB_batting', 'HBP_batting', 'IBB_batting', 'SO_batting', 'SH_batting', 'ROE_batting', 'GDP_batting',
                                                                                                                                        'SB_batting', 'CS_batting', 'BA', 'OBP', 'SLG', 'OPS', 'LOB', '#', 'IP', 'H_pitching', 'ER', 'UER', 
                                                                                                                                        'BB_pitching', 'SO_pitching', 'HR_pitching', 'HBP_pitching', 'ERA', 'BF', 'Pit', 'Str', 'SB_pitching',
                                                                                                                                        'CS_pitching', 'AB_pitching', '2B_pitching', '3B_pitching', 'ROE_pitching', 'GDP_pitching', 'Gm#']].apply(pd.to_numeric)
      ##################################### DO NOT TOUCH THE ROWS ABOVE ########################################################################################################################################################

    return teamsData # return data for all teams given a year

## Creating new attributes


In [3]:
def Create_New_Attributes(teamsData, teams):    
    # loop through each teams DF to generate additional statistics
    for index, team in enumerate(teams):

        ## OFFENSIVE STATS
        teamsData[team]['Run_Margin'] = teamsData[team]['R'] - teamsData[team]['RA']
        teamsData[team]['Run_Margin_10D_Exp_MA'] = teamsData[team]['Run_Margin'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['ER_10D_MA'] = teamsData[team]['ER'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['Daily_BA'] = teamsData[team]['H_batting'] / teamsData[team]['AB_batting']
        teamsData[team]['10D_BA_MA'] = teamsData[team]['Daily_BA'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['Daily_OBP'] = (teamsData[team]['H_batting'] + teamsData[team]['BB_batting'] + teamsData[team]['HBP_batting']) / teamsData[team]['PA'] 
        singles = teamsData[team]['H_batting'] - teamsData[team]['2B_batting'] - teamsData[team]['3B_batting'] - teamsData[team]['HR_batting']
        teamsData[team]['Daily_SLG'] =  (singles + (2 * teamsData[team]['2B_batting']) + (3 * teamsData[team]['3B_batting']) + (4 *teamsData[team]['HR_batting'])) / teamsData[team]['AB_batting'] 
        # Calculation for Slugging: (1B) +( 2 x 2B) + ( 3 x 3B) + ( 4 x HR) / AB
        teamsData[team]['10D_SLG_MA'] = teamsData[team]['Daily_SLG'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['Daily_OPS'] = teamsData[team]['Daily_OBP'] + teamsData[team]['Daily_SLG']
        teamsData[team]['10D_OPS_MA'] = teamsData[team]['Daily_OPS'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['10D_SO_MA_batting'] = teamsData[team]['SO_batting'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['15D_RBI_MA'] = teamsData[team]['RBI'].rolling(window = 15).mean() # 10 day run margin MA
        teamsData[team]['7D_HR_MA'] = teamsData[team]['HR_batting'].rolling(window = 7).mean() # 10 day run margin MA
        teamsData[team]['10D_BB_MA'] = teamsData[team]['BB_batting'].rolling(window = 10).mean() # 10 day run margin MA



        ## DEFENSIVE STATS
        teamsData[team]['Daily_ERA'] = 9 * teamsData[team]['ER'] / teamsData[team]['IP']
        teamsData[team]['10D_ERA_MA'] = teamsData[team]['Daily_ERA'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['Daily_WHIP'] = teamsData[team]['BB_pitching'] + teamsData[team]['H_pitching'] / teamsData[team]['IP']
        teamsData[team]['10D_WHIP_MA'] = teamsData[team]['Daily_WHIP'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['10D_SO_MA_pitching'] = teamsData[team]['SO_pitching'].rolling(window = 10).mean() # 10 day run margin MA
        teamsData[team]['Pit/In'] = teamsData[team]['Pit'] / teamsData[team]['IP']
        teamsData[team]['7D_Pit/In_MA'] = teamsData[team]['Pit/In'].rolling(window = 10).mean() # 10 day run margin MA
    
    return teamsData
        #teamsData[team]['Daily_OPS']
        #df['%D'] = df['%K'].ewm(span=b,min_periods=b).mean()

### Adding Starting Pitcher & Bullpen Stats

In [4]:
def Get_Pitcher_Data(year, teams):
    pitching_rotation = {}

    for team in teams:

        pitching_url = 'https://www.baseball-reference.com/teams/'+ team +'/' + year + '-pitching.shtml'
        # opening the url for reading
        html = urllib.request.urlopen(pitching_url)

        # parsing the html file
        soup = BeautifulSoup(html, 'html.parser')

        table = soup.find("table", { "id" : "team_pitching" })

        pitching_rotation[team] = pd.read_html(str(table))[0]

        # Change Formatting of Name Columns In Order To Merge Dataframes
        pitching_rotation[team] = pitching_rotation[team].add_suffix('_Starting_Pitcher')  # add suffix so it does not overlap with _bating or _pitching stats
        pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
        #pitching_rotation[team][['Pitcher First Name','Pitcher Last Name']] = pitching_rotation[team]['Name_Starting_Pitcher'].loc[pitching_rotation[team]['Name_Starting_Pitcher'].str.split().str.len() == 2].str.split(expand=True)
        pitching_rotation[team]['Pitcher Last Name'] = pitching_rotation[team]['Name_Starting_Pitcher'].astype(str).str.split().str[1]
        pitching_rotation[team]['Pitcher First Name'] = pitching_rotation[team]['Name_Starting_Pitcher'].astype(str).str.split().str[0]
        pitching_rotation[team]['Pitcher First Initial'] =  pitching_rotation[team]['Name_Starting_Pitcher'].str[:1] # get first initial of pitchers first name (since teams have multiple pitchers with same last name)

        pitching_rotation[team] = pitching_rotation[team][pitching_rotation[team]['Rk_Starting_Pitcher'] != 'Rk'] # delete rows where the column statistics are repeated on the website
        pitching_rotation[team] = pitching_rotation[team][pitching_rotation[team]['Rk_Starting_Pitcher'] != 'Rk'] # delete rows where the column statistics are repeated on the website
        pitching_rotation[team].drop(pitching_rotation[team].tail(2).index, inplace = True) # drop last 2 rows which contain stats for the whole team

        pitching_rotation[team][['Rk_Starting_Pitcher','W_Starting_Pitcher', 'L_Starting_Pitcher', 'W-L%_Starting_Pitcher', 'ERA_Starting_Pitcher', 'G_Starting_Pitcher', 'GS_Starting_Pitcher',
                                 'IP_Starting_Pitcher', 'H_Starting_Pitcher', 'R_Starting_Pitcher', 'ER_Starting_Pitcher', 'HR_Starting_Pitcher', 'BB_Starting_Pitcher', 'SO_Starting_Pitcher',
                                 'HBP_Starting_Pitcher', 'WP_Starting_Pitcher', 'BF_Starting_Pitcher', 'ERA+_Starting_Pitcher', 'FIP_Starting_Pitcher', 'WHIP_Starting_Pitcher', 'H9_Starting_Pitcher',
                                 'HR9_Starting_Pitcher', 'BB9_Starting_Pitcher', 'SO9_Starting_Pitcher', 'SO/W_Starting_Pitcher']] = pitching_rotation[team][['Rk_Starting_Pitcher', 'W_Starting_Pitcher', 'L_Starting_Pitcher', 'W-L%_Starting_Pitcher', 'ERA_Starting_Pitcher', 'G_Starting_Pitcher', 'GS_Starting_Pitcher',
                                                                                                                                             'IP_Starting_Pitcher', 'H_Starting_Pitcher', 'R_Starting_Pitcher', 'ER_Starting_Pitcher', 'HR_Starting_Pitcher', 'BB_Starting_Pitcher', 'SO_Starting_Pitcher',
                                                                                                                                             'HBP_Starting_Pitcher', 'WP_Starting_Pitcher', 'BF_Starting_Pitcher', 'ERA+_Starting_Pitcher', 'FIP_Starting_Pitcher', 'WHIP_Starting_Pitcher', 'H9_Starting_Pitcher',
                                                                                                                                             'HR9_Starting_Pitcher', 'BB9_Starting_Pitcher', 'SO9_Starting_Pitcher', 'SO/W_Starting_Pitcher']].apply(pd.to_numeric)
        ###################################### DO NOT CODE ABOVE THIS LINE #############################################################################################
    
    return pitching_rotation

In [5]:
def Add_Bullpen_Data(teamsData, pitching_rotation, teams):

    for team in teams:
        # Add Bullpen Stats
        teamsData[team]['Bullpen_ERA'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['ERA_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_WHIP'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['WHIP_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_H9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['H9_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_HR9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['HR9_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_BB9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['BB9_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_SO9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['SO9_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_SO/W'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['SO/W_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_FIP'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['FIP_Starting_Pitcher'].mean()
        teamsData[team]['Bullpen_ERA+'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'RP']['ERA+_Starting_Pitcher'].mean()

        # Add Closer Stats
        teamsData[team]['Closer_ERA'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['ERA_Starting_Pitcher'].mean()
        teamsData[team]['Closer_WHIP'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['WHIP_Starting_Pitcher'].mean()
        teamsData[team]['Closer_H9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['H9_Starting_Pitcher'].mean()
        teamsData[team]['Closer_HR9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['HR9_Starting_Pitcher'].mean()
        teamsData[team]['Closer_BB9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['BB9_Starting_Pitcher'].mean()
        teamsData[team]['Closer_SO9'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['SO9_Starting_Pitcher'].mean()
        teamsData[team]['Closer_SO/W'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['SO/W_Starting_Pitcher'].mean()
        teamsData[team]['Closer_FIP'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['FIP_Starting_Pitcher'].mean()
        teamsData[team]['Closer_ERA+'] = pitching_rotation[team][pitching_rotation[team]['Pos_Starting_Pitcher'] == 'CL']['ERA+_Starting_Pitcher'].mean()


        teamsData[team][['Bullpen_ERA', 'Bullpen_WHIP', 'Bullpen_H9', 'Bullpen_HR9', 'Bullpen_BB9', 'Bullpen_SO9', 'Bullpen_SO/W', 'Bullpen_FIP',
                        'Bullpen_ERA+', 'Closer_ERA', 'Closer_WHIP', 'Closer_H9', 'Closer_HR9', 'Closer_BB9', 'Closer_SO9', 'Closer_SO/W', 'Closer_FIP', 'Closer_ERA+']] = teamsData[team][['Bullpen_ERA', 'Bullpen_WHIP', 'Bullpen_H9', 'Bullpen_HR9', 'Bullpen_BB9', 'Bullpen_SO9', 'Bullpen_SO/W', 'Bullpen_FIP',
                                                                                                                                                                                            'Bullpen_ERA+', 'Closer_ERA', 'Closer_WHIP', 'Closer_H9', 'Closer_HR9', 'Closer_BB9', 'Closer_SO9', 'Closer_SO/W', 'Closer_FIP', 'Closer_ERA+']].apply(pd.to_numeric)
            ###################################### DO NOT CODE ABOVE THIS LINE #############################################################################################

    return teamsData

In [6]:
def Add_Fielding_Data(year, teams, teamsData):
    
    fielding_data = {}

    for team in teams:

        fielding_url = 'https://www.baseball-reference.com/teams/' + team + '/' + year + '-fielding.shtml'

        # NOTE: Different method of scrapping, referenced here: https://stackoverflow.com/questions/57032340/python-beautiful-soup-cant-find-specific-table
        response = requests.get(fielding_url)

        soup = BeautifulSoup(response.text, 'html.parser')

        comments = soup.find_all(string=lambda text: isinstance(text, Comment))

        tables = []
        for each in comments:
            if 'table' in each:
                try:
                    tables.append(pd.read_html(each)[0])
                except:
                    continue

        fielding_data[team] = tables[0]

        #fielding_data[team] = fielding_data[team].head(30)

        fielding_data[team].drop(fielding_data[team].tail(5).index, inplace = True) # drop last 2 rows which contain stats for the whole team
        #print(fielding_data[team].head())
        # change attributes to fielding attributes

        fielding_data[team][['G','GS', 'CG', 'Inn', 'Ch', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood', 'RF/9', 'RF/G', 'SB', 'CS',
                                                                                     'PO']] = fielding_data[team][['G', 'GS', 'CG', 'Inn', 'Ch', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood', 'RF/9', 'RF/G', 'SB', 'CS', 'PO']].apply(pd.to_numeric)

        ###################################### DO NOT CODE ABOVE THIS LINE #############################################################################################

        # filter data to only include players who play in the field often
        fielding_data[team] = fielding_data[team][(fielding_data[team]['G'] > 30) & (fielding_data[team]['Inn'] > 100)]
        #fielding_data[team]['Team_Avg_Fld%'] = fielding_data[team]['Fld%'].mean()
        teamsData[team]['Team_Avg_Fld%'] = fielding_data[team]['Fld%'].mean()
        teamsData[team]['Team_Avg_Rdrs/yr'] = fielding_data[team]['Rdrs/yr'].mean()
        teamsData[team]['Team_Avg_Rtot/yr'] = fielding_data[team]['Rtot/yr'].mean()
        teamsData[team]['Team_Avg_Rgood'] = fielding_data[team]['Rgood'].mean()
        teamsData[team]['Team_Avg_RF/G'] = fielding_data[team]['RF/G'].mean()
        
    return teamsData

### Add Away-Team's Statistics to Gameday Statline


In [7]:
def Add_Pitcher_Stats(teamsData, pitching_rotation, teams):    
    # add starting pitching data to new data Dataframe
    newDataDF = {}

    for team in teams:

        newDataDF[team] = pd.DataFrame() # initialize team dataframe to store combined stats

        for pitcher_last_name in pitching_rotation[team]['Pitcher Last Name']:

            pitcher_stats = pitching_rotation[team][pitching_rotation[team]['Pitcher Last Name'] == pitcher_last_name] # get pitcher stats
            pitcher_games = teamsData[team][teamsData[team]['Pitcher Last Name'] == pitcher_last_name] # get games pitcher pitched in

            df = pitcher_games.merge(pitcher_stats, how = 'left' , on = ['Pitcher Last Name', 'Pitcher First Initial']) # add pitcher stats to every game he's pitched in

            newDataDF[team] =  newDataDF[team].append(df)  # append pitchers results to the teams dataframe

        #newDataDF[team] = newDataDF[team].sort_values(by = 'Gm#') # sort values so games are in order
        ##newDataDF[team] = newDataDF[team].drop_duplicates('Gm#') # delete duplicates that occur on some teams only (not sure why?)
        ##newDataDF[team].reset_index(inplace = True) # reset the index that gets messed up on the merge
    
    return newDataDF
        #pitchers_games =
        #newDataDF[team] = teamsData[team].merge(pitching_rotation[team], how = 'left', on = 'Pitcher Last Name')

In [8]:
def Get_Opponent_Data(newDataDF, teams):
    
    finalDF = {}

    # merge new data with opponent matchup data to create final dataframe
    for index, team, in enumerate(teams):
        finalDF[team] = pd.DataFrame()
        for index, opponent in enumerate(teams):

            HomeTeamDF = newDataDF[team].loc[newDataDF[team]['Opp'] == opponent]  # calculate all games faced against an opponent
            OpponentsDF = newDataDF[opponent].loc[newDataDF[opponent]['Opp'] == team] # calculate opposing teams instances of the same games

            df = HomeTeamDF.merge(OpponentsDF, on = 'Date', suffixes=(None, '_Opponent')) # merge games 


            finalDF[team] = finalDF[team].append(df)
    
    return finalDF
        #finalDF[team] = finalDF[team].sort_values('Gm#') # sort values by game number (they get unsorted during the merge)
        #finalDF[team].reset_index(inplace = True) # Reset the index since it gets messed up on merge

## Main Function

In [9]:
teams = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 
         'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA',
         'SFG', 'STL', 'TBR', 'TEX', 'TOR', 'WSN']

MLB_Data = {}
#years = ['2018', '2019', '2021']
years = ['2020', '2021']
for year in years:
    data = Web_Scrape_Team_Data(year, teams)
    data = Create_New_Attributes(data, teams)
    pitcher_data = Get_Pitcher_Data(year, teams)
    data = Add_Bullpen_Data(data, pitcher_data, teams)
    data = Add_Fielding_Data(year, teams, data)
    newData = Add_Pitcher_Stats(data, pitcher_data, teams)
    MLB_Data[year] = Get_Opponent_Data(newData, teams)

<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
 

<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
 

<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)
<ipython-input-4-f4223505fcb3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  pitching_rotation[team]['Name_Starting_Pitcher'] = pitching_rotation[team]['Name_Starting_Pitcher'].str.replace(r'-man|[^a-zA-Z ]|day IL', '')#, expand=False)


In [10]:
MLB_Data['2020']['CHW'][['Pitcher Last Name', 'Pitcher First Name_Opponent', 'Pitcher Last Name_Opponent', 'Gm#', 'Date', 'Tm', 'Opp', 'W/L', 'R', 'RA', 'Bullpen_ERA_Opponent']].sort_values(by = 'Gm#').tail()


,Pitcher Last Name,Pitcher First Name_Opponent,Pitcher Last Name_Opponent,Gm#,Date,Tm,Opp,W/L,R,RA,Bullpen_ERA_Opponent
1,Rodon,Randy,Dobnak,47.0,"Monday, Sep 14",CHW,MIN,W,3.0,1.0,3.1400
0,Rodon,Sonny,Gray,52.0,"Saturday, Sep 19",CHW,CIN,W,5.0,0.0,3.8625
3,Keuchel,Cal,Quantrill,57.0,"Thursday, Sep 24",CHW,CLE,L,4.0,5.0,3.1650
0,Cease,Kyle,Hendricks,59.0,"Saturday, Sep 26",CHW,CHC,W,9.0,5.0,3.7900
1,Rodon,Adbert,Alzolay,60.0,"Sunday, Sep 27",CHW,CHC,L,8.0,10.0,3.7900


## Combine Yearly Data From Each Team

In [41]:
Combined_Years_Data = pd.DataFrame()

for year in years:
    for team in teams:
        Combined_Years_Data = Combined_Years_Data.append(MLB_Data[year][team])
        
print(len(Combined_Years_Data))

3578


In [42]:
Combined_Years_Data['W/L'] = Combined_Years_Data['W/L'].map({'W' : 'W',
                                                             'L' : 'L',
                                                             'L-wo' : 'L',
                                                             'W-wo' : 'W'})

Combined_Years_Data.to_csv('CombinedYearsData.csv')

## Let's take a look at the newly, scraped data...

In [ ]:
for team in teams:
    print(pitching_rotation[team][['Pitcher Last Name', 'Pitcher First Name', 'Pitcher First Initial']])

In [ ]:
teamsData['CHC'].info()

In [ ]:
# descriptive statistics for numerical attributes
teamsData['CHC'].describe()

In [ ]:
 newDataDF['CHC'].to_csv('newData.csv')

## Combine All Teams' Data For Training

In [ ]:
MLB_Data = pd.DataFrame()

for team in teams:
    MLB_Data = MLB_Data.append(finalDF[team])

In [ ]:
MLB_Data['W/L'] = MLB_Data['W/L'].map({'W' : 'W',
                                       'L' : 'L',
                                       'L-wo' : 'L',
                                       'W-wo' : 'W'})

In [ ]:
MLB_Data['W/L'].value_counts()

In [ ]:
#MLB_Data.to_csv('TrainData.csv')

# Machine Learning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

train_set, test_set = train_test_split(MLB_Data, test_size = 0.2, random_state = 42)

In [ ]:
home_features = ['BA', '10D_BA_MA', '10D_SLG_MA', '10D_ERA_MA', '10D_WHIP_MA', 'cLI', '10D_SO_MA_batting', '10D_SO_MA_pitching',
                 '15D_RBI_MA', '7D_HR_MA', '10D_BB_MA', 'ERA', '7D_Pit/In_MA',
                 'W-L%_Starting_Pitcher', 'WHIP_Starting_Pitcher', 'SO/W_Starting_Pitcher', 'SO9_Starting_Pitcher', 'ERA_Starting_Pitcher',
                 'W_Starting_Pitcher', 'HR_Starting_Pitcher', 'FIP_Starting_Pitcher']

opponent_features = ['10D_BA_MA_Opponent', '10D_SLG_MA_Opponent', '10D_ERA_MA_Opponent', '10D_WHIP_MA_Opponent', 'BA_Opponent', 'cLI_Opponent',
                     '10D_SO_MA_batting_Opponent', '10D_SO_MA_pitching_Opponent', '15D_RBI_MA_Opponent', '7D_HR_MA_Opponent', '10D_BB_MA_Opponent',
                     'ERA_Opponent', '7D_Pit/In_MA_Opponent', 
                     'W-L%_Starting_Pitcher_Opponent', 'WHIP_Starting_Pitcher_Opponent', 'SO/W_Starting_Pitcher_Opponent', 'SO9_Starting_Pitcher_Opponent', 'ERA_Starting_Pitcher_Opponent',
                     'W_Starting_Pitcher_Opponent', 'HR_Starting_Pitcher_Opponent', 'FIP_Starting_Pitcher_Opponent']


all_features = home_features + opponent_features


## Create Pipeline for Standardizing Data
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('std_scaler', StandardScaler())
])

# fit transformation to training data and then transform TestData
TrainingData = num_pipeline.fit_transform(MLB_Data[all_features])
TestData = num_pipeline.transform(test_set[all_features])

# Define the Model type
#forest_reg = RandomForestRegressor()
#DT= DecisionTreeClassifier()
model = svm.SVC(probability = True)

# prepare data and labels
win_loss_labels = MLB_Data['W/L']


# fit model to data
model.fit(TrainingData, win_loss_labels)

# calculate confidence in predictions
class_probabilities = model.predict_proba(TestData)

# determine Classification
pred= model.predict(TestData)

print(class_probabilities)

In [ ]:
test_set['W/L']

In [ ]:
pred

## Predicting Todays Games

In [ ]:
import urllib.request 
from bs4 import BeautifulSoup
import re

todays_games_url = 'https://www.baseball-reference.com/leagues/MLB-schedule.shtml#today'
team_list = ['Arizona D\'Backs', 'Atlanta Braves', 'Baltimore Orioles', 'Boston Red Sox', 'Chicago White Sox', 'Chicago Cubs',
             'Cincinnati Reds', 'Cleveland Indians', 'Colorado Rockies', 'Detroit Tigers', 'Houston Astros', 'Kansas City Royals',
             'Los Angeles Angels', 'Los Angeles Dodgers', 'Miami Marlins', 'Milwaukee Brewers', 'Minnesota Twins', 'New York Yankees',
             'New York Mets', 'Oakland Athletics', 'Philadelphia Phillies','Pittsburgh Pirates', 'San Diego Padres', 'San Francisco Giants', 
             'Seattle Mariners', 'St. Louis Cardinals', 'Tampa Bay Rays', 'Texas Rangers', 'Toronto Blue Jays', 'Washington Nationals']


# opening the url for reading
html = urllib.request.urlopen(todays_games_url)

# parsing the html file
soup = BeautifulSoup(html, 'html.parser')

todays_games_DF = pd.DataFrame(columns = ['Away Team', 'Home Team'])

target = soup.find('h3', text = 'Today\'s Games')
for sib in target.find_next_siblings(['p']):
    if sib.name=="h3":
        break
    else:
        #todays_games_DF = todays_games_DF.append(sib.text)
        matchup = re.findall(r"(?=("+'|'.join(team_list)+r"))", sib.text)
        print(matchup)
        matchup_series = pd.Series(matchup, index = todays_games_DF.columns)
        todays_games_DF = todays_games_DF.append(matchup_series, ignore_index = True)

# for para in htmlParse.find_all("p"):
#     print(para.get_text())

#df = pd.read_html(todays_games_url)

# for index, team in enumerate(teams):
#     ## Collect schedule information for each team
#     baseball_schedule_url = 'https://www.baseball-reference.com/teams/' + team + '/2021-schedule-scores.shtml'
#     dfs = pd.read_html(baseball_schedule_url)
#     scheduleDF = pd.concat(dfs)

In [ ]:
abreviation_map = {'Arizona D\'Backs' : 'ARI', 'Atlanta Braves': 'ATL', 'Baltimore Orioles': 'BAL', 'Boston Red Sox': 'BOS', 'Chicago White Sox': 'CHW', 'Chicago Cubs': 'CHC',
             'Cincinnati Reds': 'CIN', 'Cleveland Indians': 'CLE', 'Colorado Rockies': 'COL', 'Detroit Tigers': 'DET', 'Houston Astros': 'HOU', 'Kansas City Royals': 'KCR',
             'Los Angeles Angels': 'LAA', 'Los Angeles Dodgers':'LAD', 'Miami Marlins': 'MIA', 'Milwaukee Brewers': 'MIL', 'Minnesota Twins': 'MIN', 'New York Yankees': 'NYY',
             'New York Mets': 'NYM', 'Oakland Athletics': 'OAK', 'Philadelphia Phillies': 'PHI','Pittsburgh Pirates': 'PIT', 'San Diego Padres': 'SDP', 'San Francisco Giants': 'SFG', 
             'Seattle Mariners': 'SEA', 'St. Louis Cardinals': 'STL', 'Tampa Bay Rays': 'TBR', 'Texas Rangers': 'TEX', 'Toronto Blue Jays': 'TOR', 'Washington Nationals': 'WSN'}

# Replace full names with abreviations
todays_games_DF.replace(abreviation_map, inplace = True)


In [ ]:
matchupData = pd.DataFrame()

# Add data for each matchup
for index, teams in todays_games_DF.iterrows():
    home_team = teams['Home Team']
    away_team = teams['Away Team']
    
    home_data = finalDF[home_team].sort_values(by = 'Gm#').tail(1)[home_features]
    home_data['Home Team'] = home_team
    
    away_data = finalDF[away_team].sort_values(by = 'Gm#').tail(1)[home_features]
    away_data = away_data.add_suffix('_Opponent')
    away_data['Away Team'] = away_team
    
    print(home_data.columns)
    print(away_data.columns)
    matchupData = matchupData.append(pd.concat([home_data, away_data], axis = 1))
    #print(matchupData)
    

In [ ]:
matchupData
#todays_games_DF
#finalDF['LAD'].sort_values(by = 'Gm#').tail(1)[home_features]
# finalDF['CHC'].sort_values(by = 'Gm#').tail(1)[home_features]
# home_data
# away_data

In [ ]:
# transform the matchup data
newData = num_pipeline.transform(matchupData[all_features])


# determine Classification
pred = model.predict(newData)

# calculate confidence in predictions
class_probabilities = model.predict_proba(newData)

matchupData['Predictions'] = pred.tolist()

#matchupData[['Home Team Win Probability', 'Away Team Win Probability']] = class_probabilities.tolist()
matchupData[['Away Team Win Probability','Home Team Win Probability',]] = class_probabilities.tolist()

print(class_probabilities)

In [ ]:
matchupData[['Home Team', 'Away Team', 'Home Team Win Probability', 'Away Team Win Probability', 'Predictions']].sort_values(by = 'Home Team Win Probability', ascending = False)

In [ ]:
matchupData[matchupData['Home Team'] == 'BAL'][['BA', '10D_BA_MA', '10D_SLG_MA', 'BA_Opponent','W-L%_Starting_Pitcher', 'WHIP_Starting_Pitcher', 'SO/W_Starting_Pitcher', '10D_BA_MA_Opponent', '10D_SLG_MA_Opponent', 'W-L%_Starting_Pitcher_Opponent', 'WHIP_Starting_Pitcher_Opponent', 'SO/W_Starting_Pitcher_Opponent']]

In [ ]:
teamsData[team]['7D_Pit/In_MA']

In [ ]:
finalDF['LAD'].tail()

In [ ]:
MLB_Data[['10D_WHIP_MA', 'BA', '10D_ERA_MA_Opponent', '10D_WHIP_MA_Opponent']]

In [ ]:
a ='asdf'
a[:1]

In [ ]:
finalDF['BAL'][['ERA_Starting_Pitcher','ERA_Starting_Pitcher_Opponent' ,'Pitcher Last Name', 'Pitcher Last Name_Opponent','Gm#','BA', '10D_BA_MA', '10D_SLG_MA', 'BA_Opponent','W-L%_Starting_Pitcher', 'WHIP_Starting_Pitcher', 'SO/W_Starting_Pitcher', '10D_BA_MA_Opponent', '10D_SLG_MA_Opponent', 'W-L%_Starting_Pitcher_Opponent', 'WHIP_Starting_Pitcher_Opponent', 'SO/W_Starting_Pitcher_Opponent']].sort_values(by = 'Gm#')

In [ ]:
print(len(finalDF))

In [11]:
teamsData['2020']['CHC']

NameError: name 'teamsData' is not defined